# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows for T6000, T5000 values
- has multiple steps
1. Backs up the raw .xlsx file created by Reach_Curator_py38_v3.py
2. Takes raw .xlsx file created by Reach_Curator_py38_v3.py --> Adds in missing rows by reading _events.txt file and comparing it --> puts this modifed version with added rows in the root folder Grant_curate
3. Once you have manually curated the session (ie. added reaches) using the Reach_Curator_py38_v3.py
4. this will back up that manually curated .xlsx file into this folder, which it creates if it does not exist already --> completed_manual_curation_backup
5. finally, it will remove any rows that have no values for ReachInit > ReachMax > ReachEnd_ --> it will then put this final cleaned version back into the root folder that the Reach_Curator_py38_v3.py saves and reads from --> Grant_curate

# ⭐ What this file does ⭐

## ✅ Results of running this file
1. you have created a duplicate of the .xlsx file that the reach_curator_py38_v3.py makes when you load in a session for the first time 
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/reach_curator_direct_backup
    - Note, this file is missing all the T6000 and T5000 rows that the find_reach_events.ipynb did not associate a reach too
2. you have created a new .xlsx file that now containes every sinlge row, meaning it added rows for every single T6000 > T5000, and left the reachInit > reachMax > reachEnd vlaues empty, this way you can manually go into the curator and add reaches for these rows
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/added_missing_rows_backup

    
### ✅ Next Steps after running this file
1. Go back to reach_Curator_p38_v3.py, and simply launch it liuke normal. then manually place all the reaches you need 

2. Once 100% done adding reachs and label. open the analyze_curated_reach_results.ipynb file (its in the same folder as this notebook)
    - then simply run that file
    - it will first back up your manaual curation .xlsx file too --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/completed_manual_curation_backup
    - it will extract / drop all empty rows from that .xlsx file and save the final version too -->  Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/final_backup
    - and it will save this final_df to the root folder, so the reach_curator can load it in --> Grant_curate/final_xlsx_file.xlsx 

In [8]:
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path


## Modify these three variables as needed

In [3]:
# Load mouse_name from .env
import os
from dotenv import load_dotenv

load_dotenv()

# Ensure that the MOUSE_NAME environment variable is set
MOUSE = os.getenv('MOUSE')
if MOUSE is None:
    raise ValueError("MOUSE_NAME environment variable is not set. Please set it to the name of the mouse.")
else:
    print(f"MOUSE loaded: {MOUSE}")
    print('\n')

print('-- First Neuropixels File --')
NP_FILE = os.getenv('NP_FILE')
if NP_FILE is None:
    raise ValueError("NP_FILE environment variable is not set. Please set it to the path of the neuropixels file.")
else:
    print(f"NP_FILE loaded: {NP_FILE}")

DATE = os.getenv('DATE')
if DATE is None:
    raise ValueError("DATE environment variable is not set. Please set it to the date of the recording.")
else:
    print(f"DATE loaded: {DATE}")

SESSION = os.getenv('SESSION')
if SESSION is None:
    raise ValueError("SESSION environment variable is not set. Please set it to the session number.")
else:
    print(f"SESSION loaded: {SESSION}")

print('\n')
print('-- Second Neuropixels File --')
NP_FILE_01 = os.getenv('NP_FILE_01')
if NP_FILE_01 is None:
    raise ValueError("NP_FILE_01 environment variable is not set. Please set it to the path of the neuropixels file.")
else:
    print(f"NP_FILE loaded: {NP_FILE_01}")

DATE_01 = os.getenv('DATE_01')
if DATE_01 is None:
    raise ValueError("DATE_01 environment variable is not set. Please set it to the date of the recording in MMDD format.")
else:
    print(f"DATE_01 loaded: {DATE_01}")

SESSION_01 = os.getenv('SESSION_01')
if SESSION_01 is None:
    raise ValueError("SESSION_01 environment variable is not set. Please set it to the session number in MMDD format.")
else:
    print(f"SESSION_01 loaded: {SESSION_01}")

print('\n')
print('-- Third Neuropixels File --')
NP_FILE_02 = os.getenv('NP_FILE_02')
if NP_FILE_02 is None:
    raise ValueError("NP_FILE_02 environment variable is not set. Please set it to the path of the second neuropixels file.")
else:
    print(f"NP_FILE_02 loaded: {NP_FILE_02}")

DATE_02 = os.getenv('DATE_02')
if DATE_02 is None:
    raise ValueError("DATE_02 environment variable is not set. Please set it to the date of the recording in MMDD format.")
else:
    print(f"DATE_02 loaded: {DATE_02}")

SESSION_02 = os.getenv('SESSION_02')
if SESSION_02 is None:
    raise ValueError("SESSION_02 environment variable is not set. Please set it to the session number in MMDD format.")
else:
    print(f"SESSION_02 loaded: {SESSION_02}")


print('\n')
print('-- Behavioral Files --')
BEHAVIORAL_FOLDER = os.getenv('BEHAVIORAL_FOLDER')
if BEHAVIORAL_FOLDER is None:
    raise ValueError("BEHAVIORAL_FOLDER environment variable is not set. Please set it to the path of the behavioral recordings.")
else:
    print(f"BEHAVIORAL_FOLDER loaded: {BEHAVIORAL_FOLDER}")

# Construct the recording folder path
root_recording_folder = fr"G:\Grant\neuropixels\kilosort_recordings\{NP_FILE}"  # Replace with actual root path
recording_folder = os.path.join(root_recording_folder, "Record Node 103", "experiment1", "recording1", "continuous")

#root_behavior_path = fr"G:\Grant\neuropixels\behavioral_recordings\{BEHAVIORAL_FOLDER}"
# Example values (replace as needed)A
root_behavior_path = fr"G:\Grant\behavior_data\DLC_net\{BEHAVIORAL_FOLDER}"
behavioral_folder = os.path.join(root_behavior_path, DATE, SESSION)

# Check if the recording folder exists
if not os.path.exists(recording_folder):
    raise FileNotFoundError(f"Recording folder not found: {recording_folder}")
else:
    print("Neuropixel Recording folder found:", recording_folder)

MOUSE loaded: reach3


-- First Neuropixels File --
NP_FILE loaded: reach3_01_2024-11-14_21-30-18_001
DATE loaded: 20241114
SESSION loaded: session010


-- Second Neuropixels File --
NP_FILE loaded: reach3_02_2024-11-14_23-57-24_002
DATE_01 loaded: 20241114
SESSION_01 loaded: session011


-- Third Neuropixels File --
NP_FILE_02 loaded: NA
DATE_02 loaded: NA
SESSION_02 loaded: NA


-- Behavioral Files --
BEHAVIORAL_FOLDER loaded: grant_reach3_swingDoor-christie
Neuropixel Recording folder found: G:\Grant\neuropixels\kilosort_recordings\reach3_01_2024-11-14_21-30-18_001\Record Node 103\experiment1\recording1\continuous


### Load in intermeidate files 
- df_closedLoop_stim_loaded --> this file has all the closed loop optical time stamps aligned to each trial, providing both the Neuropixel timestamp And the actual associated frame number  
1. "T5000", "closed_loop_stim" --> are the Neuropixel Timestamps 
2. "T5000_frame" --> is the associated frame number 
3. "stim_count" --> is the optical stim number, should go up by 10 each row, assuming you did 10 optical pulses per trail
4. "stim_present" --> is self-explanitory, True if there was a stim for that trial, NaN if there was not


In [28]:

print(f'Note: if this cell fails to load in the DF, you can assume it did not save')
int_dir   = Path(root_recording_folder) / 'intermediates'
file_path = int_dir / 'T5000_closedLoop_stim_df.csv'

# now load:
df_closedLoop_stim_loaded = pd.read_csv(file_path)
df_closedLoop_stim_loaded.head()

Note: if this cell fails to load in the DF, you can assume it did not save


,T5000,closed_loop_stim,stim_count,stim_present,T5000_frame
0,2188.344875,2188.856325,0,True,1829
1,2202.343700,2203.343900,10,True,4002
2,2216.854500,2217.345475,20,True,6102
3,2230.595325,2231.092400,30,True,8164
4,2246.842125,2247.347050,40,True,10601


In [29]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net

root_folder = BEHAVIORAL_FOLDER # change this to your root folder for one specfic mouse (this should contain multiple sessions)


SESSION_OPTIONS = [f"{DATE}_{SESSION}", f"{DATE_01}_{SESSION_01}", f"{DATE_02}_{SESSION_02}"]
# Set the specific mouse session to analyze

session_info = SESSION_OPTIONS[0] # change this to the session you want to analyze
print(session_info)

20241114_session010


### Create new session_option for each session
- class that lets you load data for specific mouse sesssion

In [30]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

class select_mouse_session:
    def __init__(self, mouse, session_date, session_ID):
        self.mouse = mouse
        self.session_date = session_date
        self.session_ID = session_ID

    def __repr__(self):
        return f"Mouse: {self.mouse}, Session Date: {self.session_date}, Session ID: {self.session_ID}"
    
    def get_mouse_session_files(self):
        mouse_session = f"{self.mouse}_{self.session_date}_{self.session_ID}"
        xlsx_file = rf'{main_path}\Grant_curate\{self.session_date}_christielab_{self.session_ID}.xlsx'
        txt_file = rf"{main_path}\videos\{self.session_date}\christielab\{self.session_ID}\{self.session_date}_christielab_{self.session_ID}_events_shifted.txt"
        xlsx_orig = rf'{main_path}\Grant_curate\xlsx_backups\curator_direct_backup\{self.session_date}_christielab_{self.session_ID}.xlsx'
        
        return xlsx_file, txt_file, xlsx_orig
    
    def get_mouse_info(self):
        mouse_info = {
            'mouse': self.mouse,
            'session_date': self.session_date,
            'session_ID': self.session_ID
        }
        return mouse_info
    

# Strip the session info to get the session date and ID
session_date = session_info.split('_')[0]
session_ID = session_info.split('_')[1]

# Strip the mouse name from the root folder
mouse = root_folder.split('_')[1]

# Create an instance of the class with the mouse name, session date, and session ID
class_intsance = select_mouse_session(mouse=mouse, session_date=session_date, session_ID=session_ID)

# Get the mouse session files and info
xlsx_file, txt_file, xlsx_orig = class_intsance.get_mouse_session_files()
mouse_info = class_intsance.get_mouse_info()

# Print the results
print(f'mouse_info: {mouse_info}')
print(f'xlsx_file: {xlsx_file}')

main_path: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie
mouse_info: {'mouse': 'reach3', 'session_date': '20241114', 'session_ID': 'session010'}
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\20241114_christielab_session010.xlsx


In [31]:
session_date = mouse_info['session_date']
session_ID = mouse_info['session_ID']
mouse = mouse_info['mouse']
session_save_name = f"{mouse}_{session_date}_{session_ID}"
print('=========================')
print(f'session_save_name: {session_save_name}')
print('')
print(f'mouse: {mouse}')
print(f'session_date: {session_date}')
print(f'session_ID: {session_ID}')


    # ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name,'reach_curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)
print(f'original_dir: {original_dir}')
print(f'backup_dir: {backup_dir}')


session_save_name: reach3_20241114_session010

mouse: reach3
session_date: 20241114
session_ID: session010
original_dir: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\reach_curator_direct_backup


## Checking if you have already analyzed this session
- if you have, the code is going to stop you from re-analyzing it by KILLING the keneral
- you you do want to overwrite these files --> you need to re-run from the top and skip this cell

In [19]:
final_df_backup = os.path.join(os.path.dirname(xlsx_file), 'xlsx_backups',session_save_name ,'final_backup')
final_df_backup_file_path = os.path.join(final_df_backup, os.path.basename(xlsx_file))

manual_curation_backup = os.path.join(os.path.dirname(xlsx_file),'xlsx_backups', session_save_name,'completed_manual_curation_backup')
manual_curation_backup = os.path.join(manual_curation_backup, os.path.basename(xlsx_file))

if os.path.exists(manual_curation_backup):
    print('⚠️ Looks like you have already completed the manual curation for this session ⚠️')
    print('❌ Therefore, in an effort to prevent overwriting any files, the script changed is going to KILL the kernal (sorry)❌')
    print('')
    print(f'manual_curation_backup: {manual_curation_backup}')
    print('')
    print('===========================================')
    print(' ✅ if you actually want to run this script on this session then simply skip running this cell next time ✅')
    print('===========================================')
    exit()

print('')
if os.path.exists(final_df_backup_file_path):
    print('⚠️ Looks like you have already completed the final curation for this session ⚠️')
    print('❌ Therefore, in an effort to prevent overwriting any files, the script changed is going to KILL the kernal (sorry)❌')
    print('')
    print(f'final_df_backup_file_path: {final_df_backup_file_path}')
    print('')
    print('===========================================')
    print(' ✅ if you actually want to run this script on this session then simply skip running this cell next time ✅')
    print('===========================================')    
    exit()

print('✅ Looks like you have not completed the first step for this session yet ✅')
print('⭐=============== Therefore you should continue running this file =======================⭐')



✅ Looks like you have not completed the first step for this session yet ✅
⭐=============== Therefore you should continue running this file =======================⭐


## Verify existence of session files

In [20]:
if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")
    print('--> You need to run this behavioral session with the newest version of find_reach_events.py first to create the txt file')
    print('\n')

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
if os.path.exists(backup_file_path):
    print(f"⚠️Warning: Backup file already exists at {backup_file_path}⚠️")
else:
        # Check if files exist
    if not os.path.exists(xlsx_file):
        print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
        print('--> You need to open this behavioral session with reach_curator_v3.py first to create the xlsx file')
        print('\n')
    else:
        # Copy the original file to the backup directory
        shutil.copy2(xlsx_file, backup_file_path)
        print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...

print(f'backup_dir: {backup_file_path}')
print('=========================')
print(f'xlsx_file: {xlsx_file}')
print('=========================')
print(f'txt_file: {txt_file}')
print('=========================')
print(f'xlsx_orig: {xlsx_orig}')


⚠️Warning: Backup file already exists at G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\reach_curator_direct_backup\20241114_christielab_session010.xlsx⚠️
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\reach_curator_direct_backup\20241114_christielab_session010.xlsx
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\20241114_christielab_session010.xlsx
txt_file: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\videos\20241114\christielab\session010\20241114_christielab_session010_events_shifted.txt
xlsx_orig: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\curator_direct_backup\20241114_christielab_session010.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [21]:
# Load existing curated xlsx
print(f"Loading existing xlsx file: {xlsx_file}")
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

Loading existing xlsx file: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\20241114_christielab_session010.xlsx


,T6000,T5000,Added_row,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected,ReachType,pellet_present
0,1744,1829,1829.0,1868.0,1882.0,1895.0,0,missed,1826,1900.0,short,1.0
1,3843,4002,4002.0,4058.0,4075.0,4106.0,0,dropped,3999,4076.0,short,1.0
2,6019,6103,6103.0,6143.0,6158.0,6731.0,0,dropped,6100,6177.0,on_pellet,1.0
3,8081,8165,8165.0,8210.0,8223.0,8235.0,0,stalled,8162,8237.0,short,1.0
4,8081,8165,8165.0,8362.0,8373.0,10406.0,0,dropped,8162,8237.0,on_pellet,1.0


In [152]:

class modify_curator_xlsx_file(object):
    def __init__(self, xlsx_file, txt_file,save_df=True):
        self.txt_file = txt_file
        self.xlsx_file = xlsx_file
        self.save_df = save_df

        
    def create_new_xlxs_file_v1(self):
        save_df = self.save_df
        # Load the existing xlsx file
        df_orig = pd.read_excel(self.xlsx_file)
        # Check if the file is empty
        txt_file = self.txt_file
        if not os.path.exists(txt_file):
            print(f"⚠️Warning: {txt_file} does not exist⚠️")
            return None
        # Check if the xlsx file is empty
        xlsx_file = self.xlsx_file
        if not os.path.exists(xlsx_file):
            print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
            return None

        if df_orig.empty:
            print("⚠️Warning: The xlsx file is empty⚠️")
            return None
        
        # Normalize column names just in case
        df_orig.columns = df_orig.columns.str.strip()

        # Load and parse txt as done earlier
        events = []
        current_event = {}
        with open(txt_file, 'r') as f:
            for line in f:
                if line.strip():
                    key, value = line.strip().split('\t')
                    value = int(value)
                    if key == 'T6000_played':
                        if current_event:
                            events.append(current_event)
                        current_event = {'T6000': value}
                    elif key == 'pellet_detected':
                        current_event['pellet_detected'] = value
                    elif key == 'pellet_delivery':
                        current_event['pellet_delivery'] = value
                    elif key == 'T5000_played':
                        current_event['T5000'] = value
        if current_event:
            events.append(current_event)

        events_df = pd.DataFrame(events)

        # Find which T6000s are missing from original file
        existing_T6000s = df_orig['T6000'].values
        new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

        # Fill in missing columns to match structure
        new_rows['reachInit'] = np.nan
        new_rows['reachMax'] = np.nan
        new_rows['reachEnd'] = np.nan
        new_rows['stim'] = 0
        new_rows['behaviors'] = np.nan
        new_rows['ReachType'] = np.nan

        # # Reorder columns to match original file
        # new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
        #                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

        # Reorder columns to match original file
        new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected','ReachType']]

        # Combine and sort by T6000
        combined = pd.concat([df_orig, new_rows], ignore_index=True)
        combined = combined.sort_values(by='T6000').reset_index(drop=True)

        # Force final column order
        combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim', 'behaviors',
                            'pellet_delivery', 'pellet_detected','ReachType']]


        # ---- Step 2: Save output as the same original file (overwrite it)
        xlsx_save_path = xlsx_file
        self.xlsx_file = xlsx_save_path

        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir,'xlsx_backups',session_save_name ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        backup_added_rows = os.path.join(original_dir, 'xlsx_backups' ,session_save_name,'added_missing_rows_backup')
        os.makedirs(backup_added_rows, exist_ok=True)
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        # Save the combined DataFrame to the same xlsx file
        if save_df:
            if os.path.exists(backup_file_path):
                combined.to_excel(xlsx_save_path, index=False)
                combined.to_excel(backup_added_rows_file_path, index=False)
                print(f"✅ Final merged Excel saved to: {xlsx_save_path}")
                print(f"✅ Backup saved to: {backup_added_rows_file_path}")
                return combined, xlsx_save_path
            else:
                print(f"⚠️Warning: Backup file doesnt exists⚠️")
                print('❌ Stopping the script to avoid overwriting the xlsx file without a back up❌')
                print(f"⚠️Warning: {backup_file_path} does not exist⚠️")
                return None, None
        else:
            print(f"⚠️Warning: DataFrame not saved to {xlsx_save_path}⚠️")
            print('set save_df to True to save the DataFrame')
            return combined
    
    def create_new_xlxs_file(self):
        save_df = self.save_df
        txt_file = self.txt_file
        xlsx_file = self.xlsx_file

        # 1) Validate TXT and XLSX existence
        if not os.path.exists(txt_file):
            print(f"⚠️ Warning: {txt_file} does not exist")
            return None, None
        if not os.path.exists(xlsx_file):
            print(f"⚠️ Warning: {xlsx_file} does not exist")
            return None, None

        # 2) Load original Excel into DataFrame
        df_orig = pd.read_excel(xlsx_file)
        if df_orig.empty:
            print("⚠️ Warning: The xlsx file is empty")
            return None, None
        df_orig.columns = df_orig.columns.str.strip()
        # Mark all original rows
        df_orig['Added_row'] = ''

        # 3) Parse event TXT file
        events = []
        current_event = {}
        with open(txt_file, 'r') as f:
            for line in f:
                if line.strip():
                    key, val = line.strip().split('\t')
                    value = int(val)
                    if key == 'T6000_played':
                        if current_event:
                            events.append(current_event)
                        current_event = {'T6000': value}
                    elif key == 'pellet_detected':
                        current_event['pellet_detected'] = value
                    elif key == 'pellet_delivery':
                        current_event['pellet_delivery'] = value
                    elif key == 'T5000_played':
                        current_event['T5000'] = value
        if current_event:
            events.append(current_event)
        events_df = pd.DataFrame(events)

        # 4) Identify and build new rows
        existing_T6000s = df_orig['T6000'].values
        new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()
        # Initialize required columns
        for col in ['reachInit','reachMax','reachEnd','behaviors','Reach_type']:
            new_rows[col] = np.nan if 'reach' in col or col in ['behaviors'] else new_rows.get(col, np.nan)
        new_rows['stim'] = 0
        # Flag added rows with T5000
        new_rows['Added_row'] = new_rows['T5000']

        # 5) Enforce final column order
        cols = [
            'T6000','T5000','Added_row',
            'reachInit','reachMax','reachEnd',
            'stim','behaviors','pellet_delivery',
            'pellet_detected','Reach_type'
        ]
        df_orig = df_orig[cols]
        new_rows = new_rows[cols]

        # 6) Combine, sort, and reset index
        combined = pd.concat([df_orig, new_rows], ignore_index=True)
        combined = combined.sort_values('T6000', ignore_index=True)

        # 7) Backup paths
        base_dir = os.path.dirname(xlsx_file)
        primary_backup_dir = os.path.join(base_dir, 'xlsx_backups', session_save_name, 'reach_curator_direct_backup')
        primary_backup_file = os.path.join(primary_backup_dir, os.path.basename(xlsx_file))
        added_backup_dir = os.path.join(base_dir, 'xlsx_backups', session_save_name, 'added_missing_rows_backup')
        os.makedirs(added_backup_dir, exist_ok=True)
        added_backup_file = os.path.join(added_backup_dir, os.path.basename(xlsx_file))

        # 8) Save combined DataFrame and backups
        if save_df:
            if os.path.exists(primary_backup_file):
                combined.to_excel(xlsx_file, index=False)
                combined.to_excel(added_backup_file, index=False)
                print(f"✅ Final merged Excel saved to: {xlsx_file}")
                print(f"✅ Added-rows backup saved to: {added_backup_file}")
                return combined, xlsx_file
            else:
                print(f"⚠️ Warning: Primary backup missing at {primary_backup_file}")
                print("❌ Aborting: primary backup required before overwrite")
                return None, None
        else:
            print(f"⚠️ Warning: save_df is False—DataFrame created but not saved to {xlsx_file}")
            return combined, None

    def drop_empty_reach_rows(self):
      
        df = pd.read_excel(self.xlsx_file)
        # back up the newly created xlsx file
        backup_dir_02 = os.path.join(os.path.dirname(self.xlsx_file),'xlsx_backups', session_save_name,'completed_manual_curation_backup')
        os.makedirs(backup_dir_02, exist_ok=True)
        backup_file_path_02 = os.path.join(backup_dir_02, os.path.basename(self.xlsx_file))
        if os.path.exists(backup_file_path_02):
            print(f"⚠️Warning: Backup file already exists at {backup_file_path_02}⚠️")
            print('❌ Stopping the script to avoid overwriting the backup file ❌')
            return None 
        else:
            # Copy the original file to the backup directory
            shutil.copy2(self.xlsx_file, backup_file_path_02)
            print(f"✅ Backup saved to: {backup_file_path_02}")
        """
        Drop specified rows from the DataFrame.
        """
        # Drop bad rows
        df = df.dropna(subset=['reachInit']).reset_index(drop=True)
         # save the DataFrame to the same xlsx file
        if self.save_df:
            df.to_excel(self.xlsx_file, index=False)
            print(f"✅ Final merged Excel saved to: {self.xlsx_file}")   
              
        return df
    
    def load_orig_xlsx_file(self):
        """
        Load the original xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        df_orig = pd.read_excel(backup_file_path)
        return df_orig, backup_file_path
    
    def load_added_rows_xlsx(self):
        """
        Load the added rows xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_added_rows = os.path.join(original_dir,'xlsx_backups' ,session_save_name, 'added_missing_rows_backup')
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        df_added_rows = pd.read_excel(backup_added_rows_file_path)
        return df_added_rows, backup_added_rows_file_path
    
    
    def load_final_xlsx(self):
        """
        Load the combined xlsx file.
        """
        df_final = pd.read_excel(self.xlsx_file)
        return df_final, self.xlsx_file
    

    def add_optical_stim(self, frame_tol=1125):
        """
        Compare the master .xlsx’s T5000 column to df_closedLoop_stim_loaded’s
        T5000_frame, and mark stim_present=True whenever a match falls within ±tol.
        """
        # 1) load master sheet
        main_df = pd.read_excel(self.xlsx_file)

        # 2) make sure your closed-loop-stim DataFrame is available
        #    here I assume df_closedLoop_stim_loaded is in scope already
        closed_df = df_closedLoop_stim_loaded[['T5000_frame', 'stim_present']].copy()

        # 3) sort both by the key columns (required by merge_asof)
        main_df   = main_df.sort_values('T5000').reset_index(drop=True)
        closed_df = closed_df.sort_values('T5000_frame').reset_index(drop=True)

        # 4) nearest‐join within tolerance
        merged = pd.merge_asof(
            main_df,
            closed_df,
            left_on='T5000',
            right_on='T5000_frame',
            tolerance=frame_tol,
            direction='nearest'
        )

        # 5) drop the helper column and return
        merged = merged.drop(columns='T5000_frame')


        # back up the most recent xlsx file
        backup_dir = os.path.join(os.path.dirname(self.xlsx_file),'xlsx_backups', session_save_name,'closedLoop_stim_backup')
        os.makedirs(backup_dir, exist_ok=True)
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))
        if os.path.exists(backup_file_path):
            print(f"⚠️Warning: Backup file already exists at {backup_file_path}⚠️")
            print('❌ Stopping the script to avoid overwriting the backup file ❌')
            return None 
        else:
            # Copy the current original file to the backup directory
            shutil.copy2(self.xlsx_file, backup_file_path)
            print(f"✅ Backup saved to: {backup_file_path}")

         # save the DataFrame to the main xlsx file location
        merged.to_excel(self.xlsx_file, index=False)
        print(f"✅ Final closed loop optical stim df saved to: {self.xlsx_file}")   
              
        return merged

    
    def add_optical_stim_to_raw_added_rows_df(self, frame_tol=1125):
            """
            Compare the master .xlsx’s T5000 column to df_closedLoop_stim_loaded
            T5000_frame, and mark stim_present=True whenever a match falls within ±tol.
            """
            # 1) load master sheet
            main_df = pd.read_excel(self.xlsx_file)

            original_dir = os.path.dirname(self.xlsx_file)
            backup_added_rows = os.path.join(original_dir,'xlsx_backups' ,session_save_name, 'added_missing_rows_backup')
            backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

            df_added_rows = pd.read_excel(backup_added_rows_file_path)


            # 2) make sure your closed-loop-stim DataFrame is available
            #    here I assume df_closedLoop_stim_loaded is in scope already
            closed_df = df_closedLoop_stim_loaded[['T5000_frame', 'stim_present']].copy()

            # 3) sort both by the key columns (required by merge_asof)
            df_added_rows   = df_added_rows.sort_values('T5000').reset_index(drop=True)
            closed_df = closed_df.sort_values('T5000_frame').reset_index(drop=True)

            # 4) nearest‐join within tolerance
            merged = pd.merge_asof(
                df_added_rows,
                closed_df,
                left_on='T5000',
                right_on='T5000_frame',
                tolerance=frame_tol,
                direction='nearest'
            )

            # 5) drop the helper column and return
            merged = merged.drop(columns='T5000_frame')


            # 6) saved the merged 
            closedLoop_stim_root_dir = os.path.join(os.path.dirname(self.xlsx_file),'xlsx_backups', session_save_name,'closedLoop_stim')
            closedLoop_stim_on_added_rows_root_dir = os.path.join(closedLoop_stim_root_dir,'raw_added_rows_version')
            closedLoop_stim_on_added_rows_dir = os.path.join(closedLoop_stim_on_added_rows_root_dir ,os.path.basename(self.xlsx_file))
            if os.path.exists(closedLoop_stim_root_dir):
                print(f'closedLoop_stim_dir Exists Already --> {closedLoop_stim_root_dir}')
            else:
                os.makedirs(closedLoop_stim_root_dir, exist_ok=True)
                print(f'Created closedLoop_stim_dir --> {closedLoop_stim_root_dir}')

            if os.path.exists(closedLoop_stim_on_added_rows_root_dir):
                print(f'Skipping Save to stop overwrite: because closedLoop_stim_df already exists --> {closedLoop_stim_on_added_rows_root_dir} ')
            else:
                os.makedirs(closedLoop_stim_on_added_rows_root_dir, exist_ok=True)
                merged.to_excel(closedLoop_stim_on_added_rows_dir,index=False)
                print(f'Saved out closed loop optical stim df --> {closedLoop_stim_on_added_rows_root_dir}')

            return merged







In [45]:
safe_fail_modify = True

In [15]:
if safe_fail_modify:
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    df_modied, xlsx_saved_file = mouse_sessions.create_new_xlxs_file()
    df_orig, xlsx_orig_file = mouse_sessions.load_orig_xlsx_file()
    df_added_rows, xlsx_added_rows_file = mouse_sessions.load_added_rows_xlsx()
    df_final, xlsx_final_file = mouse_sessions.load_final_xlsx()
    df_modied
else:
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print('if you want to run the code, please re-run this cell')
    print('Note this ')
    safe_fail_modify = True

✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\20241114_christielab_session010.xlsx
✅ Added-rows backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\added_missing_rows_backup\20241114_christielab_session010.xlsx


### Add in closed loop optical stimulation
- adds the stim_present coulmn to the .xlsx file
- puts a 1 if there was a closed loop optical stim that trial 
#### Bonus you can run this at any point
- meaning if you already started adding reaches with the curator it will work, because it simply matches the frame numbers 


In [158]:
safe_fail_modify = True

In [159]:
if safe_fail_modify:
    # View current main .xlsx file
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    closedLoop_df = mouse_sessions.add_optical_stim()
    closedLoop_df
else:
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print(' This will add in the pellet_present coulmn to the df --> this only works if you already created the df_closedLoop_stim_loaded df when making the NWB with analyze_data/01_event_plots_makeNWB.ipynb')
    print('if you want to run the code, please re-run this cell')
    print('Note this ')
    safe_fail_modify = True

⚠️Warning: Backup file already exists at G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\closedLoop_stim_backup\20241114_christielab_session010.xlsx⚠️
❌ Stopping the script to avoid overwriting the backup file ❌


## Verify Data

In [161]:
# Load existing curated xlsx
df_added_missing_rows = pd.read_excel(xlsx_saved_file)
df_added_missing_rows.head(5)

NameError: name 'xlsx_saved_file' is not defined

In [162]:
df_orig, backup_file_path = mouse_sessions.load_orig_xlsx_file()
df_added_missing_rows, backup_added_rows_file_path = mouse_sessions.load_added_rows_xlsx()

In [164]:
orig_reach_count = df_orig.shape[0]
added_row_count = df_added_missing_rows.shape[0]
print('------')
print(f'loading orginal_df for reach events from --> {backup_file_path}')
print(f'loading df with missing rows added from--> {backup_added_rows_file_path}')
# Count the number of reach events
print('')
print(f"Original reach events total count: {len(df_orig)}")
print(f"Added reach events total count: {len(df_added_missing_rows)}")
print('')
print(f'❌ Reaches Missed: {added_row_count- orig_reach_count}')


------
loading orginal_df for reach events from --> G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\reach_curator_direct_backup\20241114_christielab_session010.xlsx
loading df with missing rows added from--> G:\Grant\behavior_data\DLC_net\grant_reach3_swingDoor-christie\Grant_curate\xlsx_backups\reach3_20241114_session010\added_missing_rows_backup\20241114_christielab_session010.xlsx

Original reach events total count: 7
Added reach events total count: 419

❌ Reaches Missed: 412


In [165]:
df_orig.head(15)

,reachInit,reachMax,reachEnd,stim,pellet_delivery,pellet_detected,T6000,T5000,behaviors,Reach_type,Added_row
0,86819,86835,86846,0,85455,85529,85374,85457,grabbed,0,NaN
1,287050,287067,287082,0,286360,270230,286199,286369,dropped,0,NaN
2,328788,328805,328814,0,327072,270230,326936,327080,dropped,0,NaN
3,342256,342270,342275,0,340424,270230,340344,340424,dropped,0,NaN
4,475419,0,0,0,473492,473570,473388,473495,none,0,NaN
5,538682,0,0,0,534860,534934,538642,534862,none,0,NaN
6,1004054,0,0,0,1002078,1002153,1001998,1002080,none,0,NaN


In [20]:
df_added_missing_rows.head(15)

,T6000,T5000,Added_row,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected,Reach_type
0,1744,1829,1829.0,NaN,NaN,NaN,0,NaN,1826,1900.0,NaN
1,3843,4002,4002.0,NaN,NaN,NaN,0,NaN,3999,4076.0,NaN
2,6019,6103,6103.0,NaN,NaN,NaN,0,NaN,6100,6177.0,NaN
3,8081,8165,8165.0,NaN,NaN,NaN,0,NaN,8162,8237.0,NaN
4,10519,10602,10602.0,NaN,NaN,NaN,0,NaN,10600,10676.0,NaN
5,13143,13228,13228.0,NaN,NaN,NaN,0,NaN,13226,13301.0,NaN
6,15845,15926,15926.0,NaN,NaN,NaN,0,NaN,15924,16000.0,NaN
7,18168,18252,18252.0,NaN,NaN,NaN,0,NaN,18250,18325.0,NaN
8,20645,20728,20728.0,NaN,NaN,NaN,0,NaN,20726,20800.0,NaN
9,22934,23093,23093.0,NaN,NaN,NaN,0,NaN,23091,23165.0,NaN


# ❌ END ❌


### ✅ Next Steps
1. Go back to reach_Curator_p38_v3.py, and simply launch it liuke normal. then manually place all the reaches you need 

2. Once 100% done adding reachs and label. open the analyze_curated_reach_results.ipynb file (its in the same folder as this notebook)
    - then simply run that file
    - it will first back up your manaual curation .xlsx file too --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/completed_manual_curation_backup
    - it will extract / drop all empty rows from that .xlsx file and save the final version too -->  Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/final_backup
    - and it will save this final_df to the root folder, so the reach_curator can load it in --> Grant_curate/final_xlsx_file.xlsx 


# ✅ Results of file
1. you have created a duplicate of the .xlsx file that the reach_curator_py38_v3.py makes when you load in a session for the first time 
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/reach_curator_direct_backup
    - Note, this file is missing all the T6000 and T5000 rows that the find_reach_events.ipynb did not associate a reach too
2. you have created a new .xlsx file that now containes every sinlge row, meaning it added rows for every single T6000 > T5000, and left the reachInit > reachMax > reachEnd vlaues empty, this way you can manually go into the curator and add reaches for these rows
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/added_missing_rows_backup
